In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings

warnings.filterwarnings('ignore')

In [7]:
# parameters
customer_id = '5'
formatted_attribute = 'size_diameter_cookware'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Plates & Platters"""

regex_pattern = r'(?i)(\s\d+\s?\d*.?\d*.?\s?D)|(depth)'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [8]:
metric = ' Inches'

range_string = """
 0 - 2.9
 12 - 14.9
 15 - 17.9
 18 - 20.9
 3 - 5.9
 6 - 8.9
 9 - 11.9
 """

In [9]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [10]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df

product_id external_id  \
0      817578630    66437673   
1      817192901    63232721   
2      817479876    63232714   
3     1219063828    63034868   
4     1219063828    63034868   
...          ...         ...   
2836   817217475    41706077   
2837   817478606    40674001   
2838  1212968615    69603305   
2839   817437118    67036059   
2840   817437115    67036189   

                                                                         product_name  \
0                             Royal Albert 100 Years 1900/1910/1940 3-Tier Cake Stand   
1                               Tabletops Gallery® Jacqueline 16-Piece Dinnerware Set   
2                                     Tabletops Gallery® Isla 16-Piece Dinnerware Set   
3                      Stamp Out Branch Wreath 11-Inch x 14-Inch Bamboo Cutting Board   
4                      Stamp Out Branch Wreath 11-Inch x 14-Inch Bamboo Cutting Board   
...                                                                               ...   
2836                                         OXO Tot® On-the-Go Feeding Spoon in Pink   
2837  Mind Reader Anchor K-Cup® Single Serve Coffee Drawer in Black (Holds 36 K-Cups)   
2838                                                     Typhoon Living Scale in Blue   
2839                        Noritake® Colorwave Coupe 16-Piece Dinnerware Set in Clay   
2840                           Noritake® Colorwave Rim 16-Piece Dinnerware Set in Ice   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                             Royal Albert's 100 Years Collection celebrates a century of beautifully served high tea. This 3-Tier Cake Stand features the 1900 Regency, 1910 Duchess, and 1940 English Chintz patterns for an elegant presentation Romantic design and gorgeous florals bring a beautiful bouquet of colors to your table Gleaming gold banding Fine bone china Hand wash Measures 13.8\" H Imported   
1                                                                                                                                                                                                                                                               Tabletops Gallery's Jacqueline 16-Piece Round Dinnerware Set updates your everyday dining with contemporary style. Beautifully crafted from white porcelain with a contemporary pattern, this elegant set includes service for 4 and is dishwasher safe. Brings style to everyday dining and casual entertaining Round shape is paired with a contemporary design Includes: Four 10.5\" dinner plates Four 8\" salad plates Four 5.5\" cereal bowls Four 10 oz. mugs Porcelain Microwave and dishwasher safe Imported   
2                                                                                                                                                                                                                                                        Tabletops Gallery's 

In [11]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0" - 2.9"',
 '12" - 14.9"',
 '15" - 17.9"',
 '18" - 20.9"',
 '21" or Greater',
 '3" - 5.9"',
 '6" - 8.9"',
 '9" - 11.9"',
 'n/a']

In [12]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0       []
1       []
2       []
3       []
4       []
        ..
2836    []
2837    []
2838    []
2839    []
2840    []
Name: long_desc, Length: 2841, dtype: object

In [13]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [14]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [15]:
x

product_id external_id  \
0      817578630    66437673   
1      817192901    63232721   
2      817479876    63232714   
9      817186029    62055833   
14     817207334    60970954   
...          ...         ...   
2815   817429767    65632963   
2819   817563068    65395769   
2820   817312346    65395806   
2822   817561989    65303511   
2834   817195615    16709123   

                                                                    product_name  \
0                        Royal Albert 100 Years 1900/1910/1940 3-Tier Cake Stand   
1                          Tabletops Gallery® Jacqueline 16-Piece Dinnerware Set   
2                                Tabletops Gallery® Isla 16-Piece Dinnerware Set   
9                                  NFL Los Angeles Chargers Metro Basket in Navy   
14    Mikasa® Italian Countryside Accents Fluted 16-Piece Dinnerware Set in Blue   
...                                                                          ...   
2815                                       MLB Detroit Tigers 20 qt. ORCA Cooler   
2819              Bumkins® Silicone First Feeding Set with Lid & Spoon in Marble   
2820                Bumkins® Silicone First Feeding Set with Lid & Spoon in Blue   
2822                 Over and Back® Hard Square 16-Piece Dinnerware Set in Black   
2834                         Mikasa® Swirl Square 4-Piece Place Setting in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Royal Albert's 100 Years Collection celebrates a century of beautifully served high tea. This 3-Tier Cake Stand features the 1900 Regency, 1910 Duchess, and 1940 English Chintz patterns for an elegant presentation Romantic design and gorgeous florals bring a beautiful bouquet of colors to your table Gleaming gold banding Fine bone china Hand wash Measures 13.8\" H Imported   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [11]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [12]:
split_col(x)

In [13]:
x['matches'].explode().value_counts() 

measures 6\" diameter         13
 8\" in diameter              11
measures 8\" diameter         10
 7\" diameter                  8
measures 5\" diameter          8
measures 5.39\" diameter       6
 10\" in diameter              6
measures 7\" diameter          6
measures 7.56\" diameter       6
measures 9\" diameter          6
 11\" in diameter              5
measures 4.5\" diameter        5
 14\" in diameter              5
measures 10.75\" diameter      4
measures 7.5\" diameter        4
measures 6.75\" diameter       4
measures 8.25\" diameter       3
 75\" in diameter              3
 102\" in diameter             3
measures 4.95\" diameter       3
measures 7.1\" diameter        2
measures 5 \" diameter         2
measures 9.3\" diameter        2
measures 8.5\" diameter        2
 12\" in diameter              2
 675\" in diameter             2
measures 4\" diameter          2
measures 10.13\" diameter      2
 7\" in diameter               1
 6\" in diameter               1
measures 1

In [14]:
error

NameError: name 'error' is not defined

In [15]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\"\-','',str(measure)))

In [16]:
def remove_fractions(numbers):
    if numbers.find('/') > -1:
#         numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [17]:
# x['matches'].explode().value_counts()

In [18]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))

In [19]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'\"','',str(measure)))

In [20]:
x=x[x['matches'].astype(str)!="10'"]
x=x[x['matches'].astype(str)!="12''"] 
x=x[x['matches'].astype(str)!="612”"]   
x=x[x['matches'].astype(str)!="6½"]
len(x)

161

In [21]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'\-','',str(measure)))

In [22]:
x['matches'].explode().value_counts()

8         22
7         16
6         15
5         11
10         7
9          7
5.39       6
11         6
7.56       6
4.5        5
14         5
7.5        4
6.75       4
10.75      4
4.95       3
75         3
102        3
8.25       3
4          2
12         2
10.13      2
675        2
8.5        2
9.3        2
7.1        2
10.8       1
11.25      1
6.88       1
6.5        1
8.1        1
10.5       1
9.5        1
8.3        1
11.125     1
12.25      1
7.3        1
8.88       1
3          1
13.25      1
8.75       1
9.25       1
11.22      1
Name: matches, dtype: int64

In [23]:
# x[x['matches'].astype(str)=='47218086'] #47218086

In [24]:
x=x[(x['matches'].astype(float)>1)&(x['matches'].astype(float)<110)]

In [25]:
x['matches']=x['matches'].astype(float)

In [26]:
x

product_id external_id  \
5      817185980    62018258   
10     817186030    62055840   
54     817460359    43597789   
60       3745798    42075547   
69       3745823    42075790   
103    817453148    13519404   
164    817404955    69337086   
295    817302429    64803753   
353    988564448    69600242   
361    817395733    69123948   
366    817377721    69491549   
455    817170753    45930034   
499    817161446    41504307   
506    817161445    41504291   
521    817481201    18234319   
537    817454111    16400726   
633    817408088    69513943   
634    817417205    69513939   
667    817394786    69123986   
687    817369844    61873438   
688    817185402    61832664   
689    817368350    68068073   
690   1071896352    69652189   
691    817185423    61832619   
692   1067890527    69652187   
718    817349653    67116478   
771    817672004    65053430   
772    817672003    65053423   
929    817452644    12239246   
1061   817463009    45385407   
1117   817185978    62018227   
1169   817410032    69489791   
1216   817598498    67705054   
1268   817460356    43597758   
1318   817603782    61873421   
1319   817369845    61873407   
1345   817186018    62055581   
1422     3745816    42075721   
1430   817160463    40786445   
1456   817154284    12871252   
1473  1143286228    69663366   
1488   817404946    69337079   
1489   817404952    69337055   
1583   988649653    65651759   
1662   817394920    69123962   
1664   817377722    68534615   
1665   817609168    68534585   
1666   817609169    68534554   
1667   817607915    68422165   
1669   817417204    69513941   
1672   817410033    69489794   
1684   817286392    63770582   
1707   988517078    60001825   
1716   817192777    63194289   
1727   817464288    45930010   
1746   817161444    41504284   
1836   817191749    63005974   
1871   988564447    69600227   
1880  1149608370    69637213   
1892   817415997    69513936   
1897   817603117    68109462   
1977   817471385    60792914   
2063   817161926    41749890   
2078   817231502    46050762   
2085  1109516262    69686317   
2106   817194379    17771337   
2107   817480800    17771353   
2129   817156449    16996025   
2142   817156274    16817309   
2143   817498283    41749852   
2144   817219424    16817287   
2204   817156811    17509357   
2278   817155965    16400793   
2318   817171695    46228468   
2329   817636287    69513944   
2367   817602667    68068103   
2414   817586413    66922476   
2582   988552591    69530021   
2584   817194252    64805863   
2643   817171694    46228451   
2662   817460358    43597772   
2697   817156448    16995967   
2729   817154283    12871236   
2751  1203377894    69661561   
2797   817416246    69514052   
2837   988689802    67501137   
2839   817591020    67414871   
2840   817590897    67414864   
2892   817253638    11701310   
2920   817377723    68534493   
2921   817415998    69513938   
2934   817300143    64624082   
2948   817185977    62018210   
2952   817192779    63194333   
2953   817479799    63194296   
2979   817170752    45930027   
2993   817164233    43125357   
2995   817164053    43052110   
3081  1124707908    68770631   
3121   817636165    69516869   
3132   817636232    69513942   
3138   817368961    68109516   
3145   817398912    69245459   
3156   817395704    69123955   
3196   817602668    68068097   
3212   817349646    67115273   
3252   817459219    42775065   
3361   817457935    41565926   
3372   817406743    69478234   
3480   817345173    66914389   
3542  1203377893    69661560   
3554   817321830    65838365   
3603   817368437    61873452   
3629   817177574    47676183   
3692   817636319    69513940   
3776   817369843    61873414   
3912   817171693    46228444   
3929   817165099    43597796   
3937     3745791    42075479   
3938     3745780    42075349   
3940   817160988    41196908   
3947   817467030    47096530   
4000     3745825    42075813   
4018   817154282    12871228   
4088  

# matches

In [27]:
matches=x
matches

product_id external_id  \
5      817185980    62018258   
10     817186030    62055840   
54     817460359    43597789   
60       3745798    42075547   
69       3745823    42075790   
103    817453148    13519404   
164    817404955    69337086   
295    817302429    64803753   
353    988564448    69600242   
361    817395733    69123948   
366    817377721    69491549   
455    817170753    45930034   
499    817161446    41504307   
506    817161445    41504291   
521    817481201    18234319   
537    817454111    16400726   
633    817408088    69513943   
634    817417205    69513939   
667    817394786    69123986   
687    817369844    61873438   
688    817185402    61832664   
689    817368350    68068073   
690   1071896352    69652189   
691    817185423    61832619   
692   1067890527    69652187   
718    817349653    67116478   
771    817672004    65053430   
772    817672003    65053423   
929    817452644    12239246   
1061   817463009    45385407   
1117   817185978    62018227   
1169   817410032    69489791   
1216   817598498    67705054   
1268   817460356    43597758   
1318   817603782    61873421   
1319   817369845    61873407   
1345   817186018    62055581   
1422     3745816    42075721   
1430   817160463    40786445   
1456   817154284    12871252   
1473  1143286228    69663366   
1488   817404946    69337079   
1489   817404952    69337055   
1583   988649653    65651759   
1662   817394920    69123962   
1664   817377722    68534615   
1665   817609168    68534585   
1666   817609169    68534554   
1667   817607915    68422165   
1669   817417204    69513941   
1672   817410033    69489794   
1684   817286392    63770582   
1707   988517078    60001825   
1716   817192777    63194289   
1727   817464288    45930010   
1746   817161444    41504284   
1836   817191749    63005974   
1871   988564447    69600227   
1880  1149608370    69637213   
1892   817415997    69513936   
1897   817603117    68109462   
1977   817471385    60792914   
2063   817161926    41749890   
2078   817231502    46050762   
2085  1109516262    69686317   
2106   817194379    17771337   
2107   817480800    17771353   
2129   817156449    16996025   
2142   817156274    16817309   
2143   817498283    41749852   
2144   817219424    16817287   
2204   817156811    17509357   
2278   817155965    16400793   
2318   817171695    46228468   
2329   817636287    69513944   
2367   817602667    68068103   
2414   817586413    66922476   
2582   988552591    69530021   
2584   817194252    64805863   
2643   817171694    46228451   
2662   817460358    43597772   
2697   817156448    16995967   
2729   817154283    12871236   
2751  1203377894    69661561   
2797   817416246    69514052   
2837   988689802    67501137   
2839   817591020    67414871   
2840   817590897    67414864   
2892   817253638    11701310   
2920   817377723    68534493   
2921   817415998    69513938   
2934   817300143    64624082   
2948   817185977    62018210   
2952   817192779    63194333   
2953   817479799    63194296   
2979   817170752    45930027   
2993   817164233    43125357   
2995   817164053    43052110   
3081  1124707908    68770631   
3121   817636165    69516869   
3132   817636232    69513942   
3138   817368961    68109516   
3145   817398912    69245459   
3156   817395704    69123955   
3196   817602668    68068097   
3212   817349646    67115273   
3252   817459219    42775065   
3361   817457935    41565926   
3372   817406743    69478234   
3480   817345173    66914389   
3542  1203377893    69661560   
3554   817321830    65838365   
3603   817368437    61873452   
3629   817177574    47676183   
3692   817636319    69513940   
3776   817369843    61873414   
3912   817171693    46228444   
3929   817165099    43597796   
3937     3745791    42075479   
3938     3745780    42075349   
3940   817160988    41196908   
3947   817467030    47096530   
4000     3745825    42075813   
4018   817154282    12871228   
4088  

In [28]:
matches['matches']=matches['matches'].astype(float)

In [29]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 2.9:
        return " 0 - 2.9 Inches"
        
    if num >=  12 and num <= 14.9:
        return " 12 - 14.9 Inches"
        
    if num >=  15 and num <= 17.9:
        return " 15 - 17.9 Inches"
        
    if num >=  18 and num <= 20.9:
        return " 18 - 20.9 Inches"
        
    if num >=  3 and num <= 5.9:
        return " 3 - 5.9 Inches"
        
    if num >=  6 and num <= 8.9:
        return " 6 - 8.9 Inches"
        
    if num >=  9 and num <= 11.9:
        return " 9 - 11.9 Inches"
        
    else:
        return r"n/a"



In [30]:
# matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [31]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

product_id external_id  \
5      817185980    62018258   
10     817186030    62055840   
54     817460359    43597789   
60       3745798    42075547   
69       3745823    42075790   
103    817453148    13519404   
164    817404955    69337086   
295    817302429    64803753   
353    988564448    69600242   
361    817395733    69123948   
366    817377721    69491549   
455    817170753    45930034   
499    817161446    41504307   
506    817161445    41504291   
521    817481201    18234319   
537    817454111    16400726   
633    817408088    69513943   
634    817417205    69513939   
667    817394786    69123986   
687    817369844    61873438   
688    817185402    61832664   
689    817368350    68068073   
690   1071896352    69652189   
691    817185423    61832619   
692   1067890527    69652187   
718    817349653    67116478   
771    817672004    65053430   
772    817672003    65053423   
929    817452644    12239246   
1061   817463009    45385407   
1117   817185978    62018227   
1169   817410032    69489791   
1216   817598498    67705054   
1268   817460356    43597758   
1318   817603782    61873421   
1319   817369845    61873407   
1345   817186018    62055581   
1422     3745816    42075721   
1430   817160463    40786445   
1456   817154284    12871252   
1473  1143286228    69663366   
1488   817404946    69337079   
1489   817404952    69337055   
1583   988649653    65651759   
1662   817394920    69123962   
1664   817377722    68534615   
1665   817609168    68534585   
1666   817609169    68534554   
1667   817607915    68422165   
1669   817417204    69513941   
1672   817410033    69489794   
1684   817286392    63770582   
1707   988517078    60001825   
1716   817192777    63194289   
1727   817464288    45930010   
1746   817161444    41504284   
1836   817191749    63005974   
1871   988564447    69600227   
1880  1149608370    69637213   
1892   817415997    69513936   
1897   817603117    68109462   
1977   817471385    60792914   
2063   817161926    41749890   
2078   817231502    46050762   
2085  1109516262    69686317   
2106   817194379    17771337   
2107   817480800    17771353   
2129   817156449    16996025   
2142   817156274    16817309   
2143   817498283    41749852   
2144   817219424    16817287   
2204   817156811    17509357   
2278   817155965    16400793   
2318   817171695    46228468   
2329   817636287    69513944   
2367   817602667    68068103   
2414   817586413    66922476   
2582   988552591    69530021   
2584   817194252    64805863   
2643   817171694    46228451   
2662   817460358    43597772   
2697   817156448    16995967   
2729   817154283    12871236   
2751  1203377894    69661561   
2797   817416246    69514052   
2837   988689802    67501137   
2839   817591020    67414871   
2840   817590897    67414864   
2892   817253638    11701310   
2920   817377723    68534493   
2921   817415998    69513938   
2934   817300143    64624082   
2948   817185977    62018210   
2952   817192779    63194333   
2953   817479799    63194296   
2979   817170752    45930027   
2993   817164233    43125357   
2995   817164053    43052110   
3081  1124707908    68770631   
3121   817636165    69516869   
3132   817636232    69513942   
3138   817368961    68109516   
3145   817398912    69245459   
3156   817395704    69123955   
3196   817602668    68068097   
3212   817349646    67115273   
3252   817459219    42775065   
3361   817457935    41565926   
3372   817406743    69478234   
3480   817345173    66914389   
3542  1203377893    69661560   
3554   817321830    65838365   
3603   817368437    61873452   
3629   817177574    47676183   
3692   817636319    69513940   
3776   817369843    61873414   
3912   817171693    46228444   
3929   817165099    43597796   
3937     3745791    42075479   
3938     3745780    42075349   
3940   817160988    41196908   
3947   817467030    47096530   
4000     3745825    42075813   
4018   817154282    12871228   
4088  

In [32]:
matches['Q:size_diameter_cookware'].explode().value_counts()

 6 - 8.9 Inches      81
 9 - 11.9 Inches     35
 3 - 5.9 Inches      28
 12 - 14.9 Inches     9
n/a                   6
Name: Q:size_diameter_cookware, dtype: int64

In [33]:
matches=matches[matches['Q:size_diameter_cookware'].astype(str)!='n/a']

In [34]:
matches

product_id external_id  \
5      817185980    62018258   
10     817186030    62055840   
54     817460359    43597789   
103    817453148    13519404   
164    817404955    69337086   
295    817302429    64803753   
353    988564448    69600242   
361    817395733    69123948   
366    817377721    69491549   
455    817170753    45930034   
499    817161446    41504307   
506    817161445    41504291   
521    817481201    18234319   
537    817454111    16400726   
633    817408088    69513943   
634    817417205    69513939   
667    817394786    69123986   
687    817369844    61873438   
688    817185402    61832664   
689    817368350    68068073   
690   1071896352    69652189   
691    817185423    61832619   
692   1067890527    69652187   
718    817349653    67116478   
771    817672004    65053430   
772    817672003    65053423   
929    817452644    12239246   
1061   817463009    45385407   
1117   817185978    62018227   
1169   817410032    69489791   
1216   817598498    67705054   
1268   817460356    43597758   
1318   817603782    61873421   
1319   817369845    61873407   
1345   817186018    62055581   
1430   817160463    40786445   
1456   817154284    12871252   
1473  1143286228    69663366   
1488   817404946    69337079   
1489   817404952    69337055   
1583   988649653    65651759   
1662   817394920    69123962   
1664   817377722    68534615   
1665   817609168    68534585   
1666   817609169    68534554   
1667   817607915    68422165   
1669   817417204    69513941   
1672   817410033    69489794   
1684   817286392    63770582   
1707   988517078    60001825   
1716   817192777    63194289   
1727   817464288    45930010   
1746   817161444    41504284   
1836   817191749    63005974   
1871   988564447    69600227   
1880  1149608370    69637213   
1892   817415997    69513936   
1897   817603117    68109462   
1977   817471385    60792914   
2063   817161926    41749890   
2078   817231502    46050762   
2085  1109516262    69686317   
2106   817194379    17771337   
2107   817480800    17771353   
2129   817156449    16996025   
2142   817156274    16817309   
2143   817498283    41749852   
2144   817219424    16817287   
2204   817156811    17509357   
2278   817155965    16400793   
2318   817171695    46228468   
2329   817636287    69513944   
2367   817602667    68068103   
2414   817586413    66922476   
2582   988552591    69530021   
2584   817194252    64805863   
2643   817171694    46228451   
2662   817460358    43597772   
2697   817156448    16995967   
2729   817154283    12871236   
2751  1203377894    69661561   
2797   817416246    69514052   
2837   988689802    67501137   
2839   817591020    67414871   
2840   817590897    67414864   
2892   817253638    11701310   
2920   817377723    68534493   
2921   817415998    69513938   
2934   817300143    64624082   
2948   817185977    62018210   
2952   817192779    63194333   
2953   817479799    63194296   
2979   817170752    45930027   
2993   817164233    43125357   
2995   817164053    43052110   
3081  1124707908    68770631   
3121   817636165    69516869   
3132   817636232    69513942   
3138   817368961    68109516   
3145   817398912    69245459   
3156   817395704    69123955   
3196   817602668    68068097   
3212   817349646    67115273   
3252   817459219    42775065   
3361   817457935    41565926   
3372   817406743    69478234   
3480   817345173    66914389   
3542  1203377893    69661560   
3554   817321830    65838365   
3603   817368437    61873452   
3629   817177574    47676183   
3692   817636319    69513940   
3776   817369843    61873414   
3912   817171693    46228444   
3929   817165099    43597796   
3940   817160988    41196908   
3947   817467030    47096530   
4018   817154282    12871228   
4088   988702534    68009885   
4200   817631442    69489793   
4201   817394921    69123993   
4203   817609155    68534523   
4218   817321628    65793510   
4223   817300177    64624068   
4229  

In [35]:
matches['Q:size_diameter_cookware'].explode().value_counts()

 6 - 8.9 Inches      81
 9 - 11.9 Inches     35
 3 - 5.9 Inches      28
 12 - 14.9 Inches     9
Name: Q:size_diameter_cookware, dtype: int64

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [36]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/BedBathandBeyond - {attribute} - match upload_Shawn.csv',index=False)

In [37]:
looks_good('Bed Bath & Beyond', attribute, df, matches)